# Examples of  the various scene module functions

## Define a grid of points based on lat/long/height

In [ ]:
include("../modules/geometry.jl")
include("../modules/scene.jl")
using Plots

## planetary shape constants
earth_radius=6378.137e3
earth_eccentricity=sqrt(0.00669437999015)

# target volume grid on surface defined in geodetic lat, lon, height (θϕh)
t_θ=-90:5:90 # deg
t_ϕ=-180:10:180 # deg
t_h=0:100e3:900e3 # m target heights

#try the two different methods for creating this
@time t_geo_grid=Scene.form3Dgrid_for(t_θ,t_ϕ,t_h) # using 3 nested for loops 
@time t_geo_grid=Scene.form3Dgrid_array(t_θ,t_ϕ,t_h) # using array processing

#convert lat,lon,height grid to ECEF-xyz
t_xyz_grid=Geometry.geo_to_xyz(t_geo_grid) # convert target volume from geo to xyz

println(size(t_xyz_grid))
# plot the target grid 
plotly()
scatter(t_xyz_grid[1,:]/1e3,t_xyz_grid[2,:]/1e3,t_xyz_grid[3,:]/1e3,
    xlabel = "X-ecef", ylabel="Y-ecef", zlabel="Z-ecef", 
    markersize=0.1)#display grid in 3D


## Define target volume using relative azimuth, elevation and height

In [ ]:

#define target-grid limits
el  = 0:1:30 # elevation angles (deg)
az  = -180:10:180 # azimuth angles (deg)
hgt = 0:10:200 # target heights (m)

#create grid based on az,el,hgt
@time target_grid = Scene.form3Dgrid_for(az,el,hgt) # using 3 nested for loops

#platform location in geodetic coordinates
platform_geo = [10,50,6000e3] # platform latitude, longitude, altitude [deg,deg,m]

#create peg point
γ   = 0 # deg track angle (heading), 0 deg is north, 90 deg is east
peg = Geometry.PegPoint(platform_geo[1],platform_geo[2],γ) # deg peg point is the nadir point of platform at the center of SAR aperture

ecef_target_grid = Scene.lookh_to_xyz(target_grid,platform_geo,peg) # convert target volume from relative look vector to xyz

#display 3D grid
scatter(ecef_target_grid[1,:],ecef_target_grid[2,:],ecef_target_grid[3,:], markersize=0.5)


# Define target valume based on C,H (from SCH) and azimuth angle

In [ ]:
# target volume grid on surface defined in ch (of sch) and azimuth rotation angles (rotated around platform position vector)
t_c  = 1e6:5e5:5e6 # m target positions in c of sch
t_h  = 0:1e5:5e5 # m target heights
rot_P=-20:.5:20#-60:10:60 # deg rotation angles around platform position vector

#define platform lat,lon,hgt
platform_geo = [60,-30,500e3] #(deg, deg, m)

#create Peg point
γ   = 0 # deg track angle (heading), 0 deg is north, 90 deg is east
peg = Geometry.PegPoint(platform_geo[1],platform_geo[2],γ) # peg point is the nadir point of platform at the center of SAR aperture

#create XYZ grid based on C,H,Az
t_xyz_grid_rot=Scene.chP_to_xyz_grid(t_c,t_h,rot_P,platform_geo, peg)# convert target volume from geo to xyz

#plot the ECEF grid
scatter(t_xyz_grid_rot[1,:],t_xyz_grid_rot[2,:],t_xyz_grid_rot[3,:],camera=(30,30),markersize=1)


## Define iso-range targets on an ellipsoid (zero height) for a set of azimuth angles, at a fixed look angle

In [ ]:
include("../modules/geometry.jl")
include("../modules/scene.jl")
using LinearAlgebra
using Statistics
using Plots

#platform position and velocity
P_pos = Geometry.geo_to_xyz([0, 0, 900e3]); #platform position in ECEF
P_vel = [0, 6e3, 0]; #platform velocity
that,chat,nhat = Geometry.get_tcn(P_pos, P_vel)

N_ϕ = 50; 
ϕ_range  = range(-90π/180, stop=90π/180, length=N_ϕ);

#compute pair of look angle, slant range
θ_0  = 55π/180;
ρ_0  = Geometry.get_rho(P_pos, Geometry.tcn_lvec(that, chat, nhat, θ_0, 0));

#get Target points along an iso-range line
@time Targ_vec = Scene.isorange_ellipsoid(P_pos, P_vel, collect(ϕ_range), ρ_0, θ_0);


#convert target to LLH to check height
targ_vec_geo = Geometry.xyz_to_geo(Targ_vec);

#compute slant range difference for each look vector to check
lvec = Targ_vec .- P_pos; # all the look vectors
Δρ_all = zeros(N_ϕ)
for ii=1:N_ϕ
    Δρ_all[ii] = norm(lvec[:,ii])-ρ_0; #compute slant range and subtract input
end

plotly()
p1 = plot(ϕ_range*180/π, targ_vec_geo[3,:]*1e9, xlabel="Azimuth (deg)", ylabel="Target Height (nm)") # this should be in nm
p2 = plot(ϕ_range*180/π, Δρ_all*1e6, xlabel="Azimuth (deg)", ylabel="ρ-ρ_0 (um)") # this should be in fractions of mm


#Background lat,lon,height grid in ECEF-xyz
t_xyz_grid=Geometry.geo_to_xyz(Scene.form3Dgrid_for(-90:5:90,-180:10:180,0)) # convert target volume from geo to xyz
p3 = scatter(Targ_vec[1,:,:]/1e3, Targ_vec[2,:,:]/1e3, Targ_vec[3,:,:]/1e3, xlabel="X-axis", 
    ylabel="Y-axis", zlabel="Z-axis", markersize=1)
p3 = scatter!(t_xyz_grid[1,:]/1e3, t_xyz_grid[2,:]/1e3, t_xyz_grid[3,:]/1e3, xlabel="X-axis", 
    ylabel="Y-axis", zlabel="Z-axis", markersize=.1)
Ppeg = Geometry.geo_to_xyz([Geometry.xyz_to_geo(P_pos)[1], Geometry.xyz_to_geo(P_pos)[2], 0.0])
p3 = scatter!(Ppeg[1,:]/1e3, Ppeg[2,:]/1e3, Ppeg[3,:]/1e3, markersize=2)

l = @layout [[a;b] c]
plot(p1,p2,p3, layout=l, legend=false)

## Define iso-range targets at various heights above ellipsoid for a set of azimuth angles, at a single look angle

In [ ]:
include("../modules/geometry.jl")
include("../modules/scene.jl")
using LinearAlgebra
using Statistics

#platform position and velocity
P_pos = Geometry.geo_to_xyz([0, 0, 900e3]); #platform position in ECEF
P_vel = [0, 6e3, 0]; #platform velocity

#get TCN frame from position and velocity
that,chat,nhat = Geometry.get_tcn(P_pos, P_vel)

#get slant range at azimuth 0
θ_0 = 35π/180; #look angle to evaluate
N_ϕ = 50; #no. of points along azimuth
N_h = 10; #no. of points in vertical dimension
ϕ_range = range(-45π/180, stop=45π/180, length=N_ϕ)
hgt_rng = range(1,stop=100,length=N_h);  

#look vector at zero azimuth and zero height
lhat_0 = Geometry.tcn_lvec(that, chat, nhat, θ_0, 0.0); #unit look vector 
ρ_0 = Geometry.get_rho(P_pos, lhat_0); #slant range to ellipsoid


#get Target vector for all phi and hgts
@time Targ_phi_vec = Scene.isorange_ellipsoid(P_pos, P_vel,collect(ϕ_range), collect(hgt_rng), ρ_0, θ_0);

#All the look vectors
lvec_phi = Targ_phi_vec .- P_pos; 

#compute slant range  and target height for checking 
ρ_phi = zeros(N_ϕ,N_h)
hgt_geo = zeros(N_ϕ,N_h)
for ii=1:N_ϕ
    for jj=1:N_h
        ρ_phi[ii,jj] = norm(lvec_phi[:,ii,jj])
        hgt_geo[ii,jj] = Geometry.xyz_to_geo(Targ_phi_vec[:,ii,jj])[3];
    end
end

#plot stuff
plotly()

p2 = plot(ϕ_range*180/π, (ρ_phi .- ρ_0)*1e6, ylabel="Δρ (μm)", legend=false)
p3 = plot(ϕ_range*180/π, hgt_geo, xlabel = "Azimuth (deg)", ylabel="Geodetic Heights (m)", legend=false)

p4 = scatter(Targ_phi_vec[1,:,:]/1e3, Targ_phi_vec[2,:,:]/1e3, Targ_phi_vec[3,:,:]/1e3,
    markersize=1, legend=false)
p4 = scatter!(t_xyz_grid[1,:]/1e3, t_xyz_grid[2,:]/1e3, t_xyz_grid[3,:]/1e3, markersize=.1)
Ppeg = Geometry.geo_to_xyz([Geometry.xyz_to_geo(P_pos)[1], Geometry.xyz_to_geo(P_pos)[2], 0.0])
p4 = scatter!(Ppeg[1,:]/1e3, Ppeg[2,:]/1e3, Ppeg[3,:]/1e3, markersize=2)

l = @layout [[b;c] d]
plot(p2,p3,p4,layout=l)

## Define iso-range targets at various heights above ellipsoid for a set of azimuth angles, at a set of look angles

In [ ]:
include("../modules/geometry.jl")
include("../modules/scene.jl")
using LinearAlgebra
using Statistics
using Plots

#platform position and velocity
P_pos = Geometry.geo_to_xyz([40, 0, 900e3]); #platform position in ECEF
P_vel = [0, 0, 6e3]; #platform velocity

#get slant range at azimuth 0
N_ϕ   = 50; #no. of points along azimuth
N_h   = 20; #no. of points in vertical dimension
N_θ   = 5; #no. of look angles to evaluate
ϕ_rng = range(-45π/180, stop=45π/180, length=N_ϕ)
hgt_rng = range(1,stop=100,length=N_h);  
θ_rng = range(25π/180,stop=60π/180,length=N_θ);  

#get Target vector for all ϕ, hgts and θs
@time Targ_vec = Scene.isorange_ellipsoid(P_pos, P_vel,collect(ϕ_rng), collect(hgt_rng), collect(θ_rng));


#All the look vectors
lvec_phi = Targ_vec .- P_pos; 

#compute slant range  and target height for checking 
Δρ = zeros(N_ϕ,N_h,N_θ)
hgt_geo = zeros(N_ϕ,N_h,N_θ)
#get TCN frame from position and velocity
that,chat,nhat = Geometry.get_tcn(P_pos, P_vel)
for kk=1:N_θ
    #zero height slant range for this look angle
    ρ_0 = Geometry.get_rho(P_pos, Geometry.tcn_lvec(that, chat, nhat, θ_rng[kk], 0.0)); 
    for ii=1:N_ϕ
        for jj=1:N_h
            Δρ[ii,jj,kk] = norm(lvec_phi[:,ii,jj,kk])-ρ_0
            hgt_geo[ii,jj,kk] = Geometry.xyz_to_geo(Targ_vec[:,ii,jj,kk])[3];
        end
    end
end



#plot stuff

plotly()
θ_ind = 5; #which look angle to plot
p2 = plot(ϕ_rng*180/π, Δρ[:,:,θ_ind]*1e6, ylabel="Δρ (μm)", legend=false, 
    title= string("θ = ", string(round(θ_rng[θ_ind]*180/π), "deg")))
p3 = plot(ϕ_rng*180/π, hgt_geo[:,:,θ_ind], xlabel = "Azimuth (deg)", 
    ylabel="Geodetic Heights (m)", legend=false)

#reshape the target vector, (plotting works better this way)
tvp  = reshape(Targ_vec, 3, N_ϕ*N_h*N_θ); 
#Background lat,lon,height grid in ECEF-xyz
t_xyz_grid=Geometry.geo_to_xyz(Scene.form3Dgrid_for(-90:5:90,-180:10:180,0)) # convert target volume from geo to xyz

p4 = scatter(tvp[1,:]/1e3, tvp[2,:]/1e3, tvp[3,:]/1e3,
     markersize=1, legend=false, title="All targets")
p4 = scatter!(t_xyz_grid[1,:]/1e3, t_xyz_grid[2,:]/1e3, t_xyz_grid[3,:]/1e3, markersize=.1)
Ppeg = Geometry.geo_to_xyz([Geometry.xyz_to_geo(P_pos)[1], Geometry.xyz_to_geo(P_pos)[2], 0.0])
p4 = scatter!(Ppeg[1,:]/1e3, Ppeg[2,:]/1e3, Ppeg[3,:]/1e3, markersize=2)

l = @layout [[b;c] d]
plot(p2,p3,p4, layout=l)